In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [2]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_Gillian.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')
plot_dir = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/plots_GilliansROI')


In [3]:
def rename_roi(roi):
    cleaned_roi = roi
    #cleaned_roi = roi
    sides = ['left', 'right']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'acc']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [4]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_Gillian.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI']
# Define the desired order for 'roi_name'
roi_order = ['left_ACC', 'right_ACC', 
             'left_vmpfc','right_vmpfc',
             'left_insula','right_insula', 
             'left_hippocampus', 'right_hippocampus', 
             'left_amygdala', 'right_amygdala'
]

#df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [5]:
df_fMRI

,Subject,Task,Contrast,ROI,Mean_COPE_Value,roi_name
0,1,phase2,1,right_acc,8.021027,right_acc
1,2,phase2,1,right_acc,-4.057398,right_acc
2,3,phase2,1,right_acc,-8.549090,right_acc
3,4,phase2,1,right_acc,14.535996,right_acc
4,5,phase2,1,right_acc,-0.853068,right_acc
...,...,...,...,...,...,...
50745,97,phase3,25,right_vmpfc,3.059002,right_vmpfc
50746,98,phase3,25,right_vmpfc,8.948880,right_vmpfc
50747,99,phase3,25,right_vmpfc,-3.006068,right_vmpfc
50748,100,phase3,25,right_vmpfc,-0.635653,right_vmpfc


In [6]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [7]:
df_current = df_phase2
contrasts_of_interest = [1,3]
contrasts_CSS = contrasts_of_interest[0]
contrasts_CSR = contrasts_of_interest[1]

In [8]:
df_current

,Subject,Task,Contrast,ROI,Mean_COPE_Value,roi_name,subID,Drug,drug_condition,Gender,...,ecr_q32,ecr_q33,ecr_q34,ecr_q35,ecr_q36,demo_age,demo_sex_at_birth,Anxiety,Avoidance,total_score
0,1,phase2,1,right_acc,8.021027,right_acc,N101,Placebo,0,Female,...,3,3,7,3,3,21,1,71,66,137
1,2,phase2,1,right_acc,-4.057398,right_acc,N103,Oxytocin,1,Female,...,5,3,5,1,2,23,1,60,39,99
2,3,phase2,1,right_acc,-8.549090,right_acc,N104,Oxytocin,1,Male,...,4,2,6,1,4,33,2,104,62,166
3,4,phase2,1,right_acc,14.535996,right_acc,N105,Oxytocin,1,Female,...,3,2,6,2,2,21,1,62,45,107
4,5,phase2,1,right_acc,-0.853068,right_acc,N106,Oxytocin,1,Female,...,5,1,3,1,1,22,1,45,36,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25245,98,phase2,25,right_vmpfc,-7.508455,right_vmpfc,N255,Oxytocin,1,Female,...,5,1,4,1,1,29,1,29,19,48
25246,99,phase2,25,right_vmpfc,-4.581082,right_vmpfc,N257,Placebo,0,Female,...,4,3,5,3,1,18,1,57,36,93
25247,100,phase2,25,right_vmpfc,-2.298124,right_vmpfc,N258,Placebo,0,Female,...,3,3,6,2,2,19,1,71,51,122
25248,101,phase2,25,right_vmpfc,9.913935,right_vmpfc,N259,Oxytocin,1,Female,...,3,3,3,3,3,23,1,69,64,133


In [9]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        for gender in df_current['Gender'].unique():
            for drug in df_current['Drug'].unique():
                df_tmp = df_current.loc[(df_current['roi_name'] == roi) & 
                                        (df_current['group'] == group) & 
                                        (df_current['Gender'] == gender) &
                                        (df_current['Drug'] == drug) &
                                        (df_current['Contrast'].isin(contrasts_of_interest))].copy()
                df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
                t_statistic, p_value = stats.ttest_rel(df_pivot[[contrasts_CSS]], df_pivot[[contrasts_CSR]])
                results.append({
                'roi': roi,
                'Group': group,
                'Drug': drug,
                'Gender': gender,
                't_stat': t_statistic[0],
                'p_value': p_value[0],
                'n': len(df_pivot)  # Sample size
            })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,Group,Drug,Gender,t_stat,p_value,n,p_value_corrected
4,right_acc,Controls,Placebo,Female,3.482677,0.004046,14,0.323701
38,left_acc,Controls,Placebo,Male,-3.344939,0.005272,14,0.421741
74,right_vmpfc,Patients,Placebo,Male,3.669004,0.006319,9,0.505485
0,right_acc,Patients,Placebo,Female,-0.498427,0.627193,13,1.000000
57,right_insula,Patients,Oxytocin,Female,-0.783860,0.445325,16,1.000000
...,...,...,...,...,...,...,...,...
24,left_amygdala,Patients,Placebo,Female,0.848405,0.412817,13,1.000000
23,left_vmpfc,Controls,Oxytocin,Male,0.417846,0.684100,12,1.000000
22,left_vmpfc,Controls,Placebo,Male,-0.309102,0.762142,14,1.000000
29,left_amygdala,Controls,Oxytocin,Female,-0.138423,0.892408,12,1.000000


hypothesis: brain activations should be different between CSS and CSR if extinction affect.\
For patients:\
lh-insula: female-oxytocin [lh-insula showed no differences for controls  and patient males(with/without oxytocin), but showed greater activation in CSS(than CSR) with oxytocin in female patients]\
rh-amygdala (marginal): male-oxytocin [rh-amygdala showed no differences for controls  and patient females(with/without oxytocin), but showed smaller activation in CSS with oxytocin in male patients]\
rh-vmpfc: male-placebo [rh-vmpfc showed no differences for controls  and patient females(with/without oxytocin), but showed greater activation in CSR in placebo in male patients]\
For controls: \
rh-ACC, rh-amy and rh-vmpfc(marginal): female-placebo\
lh-acc:male-placebo

In [47]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for right_acc
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     1.751
Date:                Wed, 26 Mar 2025   Prob (F-statistic):             0.0888
Time:                        09:32:57   Log-Likelihood:                -135.25
No. Observations:                 101   AIC:                             290.5
Df Residuals:                      91   BIC:                             316.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

,Condition,Effect,p_value
2,right_acc,C(Drug)[T.Placebo],0.003806
88,right_amygdala,Avoidance_centered,0.020098
21,left_vmpfc,C(group)[T.Patients],0.034957
4,right_acc,C(group)[T.Patients]:C(Drug)[T.Placebo],0.039904
24,left_vmpfc,C(group)[T.Patients]:C(Drug)[T.Placebo],0.043111
...,...,...,...
5,right_acc,C(group)[T.Patients]:C(Gender)[T.Male],0.946223
63,left_insula,C(Gender)[T.Male],0.948223
59,right_hippocampus,demo_age,0.953646
9,right_acc,demo_age,0.959191


for the entire dataset \
rh-ACC: Drug effect, group * drug and group * gender(marginal) \
Avoidance rh-amy (lh-amy, marginal)

In [48]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender)", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for right_acc
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     2.258
Date:                Wed, 26 Mar 2025   Prob (F-statistic):             0.0362
Time:                        09:33:16   Log-Likelihood:                -135.39
No. Observations:                 101   AIC:                             286.8
Df Residuals:                      93   BIC:                             307.7
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

,Condition,Effect,p_value
2,right_acc,C(Drug)[T.Placebo],0.003231
17,left_vmpfc,C(group)[T.Patients],0.035077
20,left_vmpfc,C(group)[T.Patients]:C(Drug)[T.Placebo],0.036554
4,right_acc,C(group)[T.Patients]:C(Drug)[T.Placebo],0.042535
6,right_acc,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.043717
...,...,...,...
9,left_hippocampus,C(group)[T.Patients],0.890926
22,left_vmpfc,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.927359
51,left_insula,C(Gender)[T.Male],0.940391
3,right_acc,C(Gender)[T.Male],0.960807


In [49]:
# extinction in only placebo, healthy females
results = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') &
                        (df_current['group'] == 'Controls') & 
                        (df_current['Gender'] == 'Female') & 
                        (df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
    t_statistic, p_value = stats.ttest_rel(df_pivot[[contrasts_CSS]], df_pivot[[contrasts_CSR]])
    results.append({
    'roi': roi,
    't_stat': t_statistic[0],
    'p_value': p_value[0],
    'n': len(df_pivot)  # Sample size
})
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

extinction results for right_acc
extinction results for left_hippocampus
extinction results for left_vmpfc
extinction results for left_amygdala
extinction results for left_acc
extinction results for right_hippocampus
extinction results for left_insula
extinction results for right_insula
extinction results for right_amygdala
extinction results for right_vmpfc


,roi,t_stat,p_value,n,p_value_corrected
0,right_acc,3.482677,0.004046,14,0.040463
8,right_amygdala,-2.283304,0.039874,14,0.398742
1,left_hippocampus,0.750377,0.466393,14,1.000000
2,left_vmpfc,1.163722,0.265451,14,1.000000
3,left_amygdala,0.807546,0.433880,14,1.000000
4,left_acc,1.690399,0.114773,14,1.000000
5,right_hippocampus,-0.734859,0.475474,14,1.000000
6,left_insula,1.687797,0.115281,14,1.000000
7,right_insula,0.721294,0.483500,14,1.000000
9,right_vmpfc,0.472296,0.644546,14,1.000000


In [50]:
# extinction in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

extinction results for right_acc
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     3.086
Date:                Wed, 26 Mar 2025   Prob (F-statistic):             0.0179
Time:                        09:33:19   Log-Likelihood:                -63.431
No. Observations:                  50   AIC:                             138.9
Df Residuals:                      44   BIC:                             150.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
2,right_acc,C(Gender)[T.Male],1.065519,0.360972,2.951803,0.005051,0.338027,1.793010
26,left_acc,C(Gender)[T.Male],1.043583,0.375797,2.776983,0.008031,0.286213,1.800952
1,right_acc,C(group)[T.Patients],1.026007,0.383926,2.672406,0.010522,0.252254,1.799759
49,right_amygdala,C(group)[T.Patients],-0.859911,0.421318,-2.041001,0.047277,-1.709023,-0.010800
47,right_insula,demo_age,0.039492,0.022061,1.790142,0.080314,-0.004969,0.083952
22,left_amygdala,Avoidance_centered,-0.010528,0.007304,-1.441415,0.156547,-0.025249,0.004192
27,left_acc,C(group)[T.Patients]:C(Gender)[T.Male],-0.719822,0.566844,-1.269876,0.210804,-1.862221,0.422577
3,right_acc,C(group)[T.Patients]:C(Gender)[T.Male],-0.680711,0.544482,-1.250198,0.217836,-1.778042,0.416621
25,left_acc,C(group)[T.Patients],0.482502,0.399694,1.207177,0.233814,-0.323029,1.288032
52,right_amygdala,Avoidance_centered,0.008633,0.007343,1.175597,0.246078,-0.006167,0.023432


In placebo. \
Gender in bi-ACC; \
Group in rh-ACC, rh-AMY, rh-vmpfc (marginal); 

In [51]:
# extinction in each group
results_list = []
drug_results_list = []
for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('p', ascending=True)
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for right_acc Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                 -0.082
Method:                 Least Squares   F-statistic:                    0.2720
Date:                Wed, 26 Mar 2025   Prob (F-statistic):              0.926
Time:                        09:33:22   Log-Likelihood:                -68.765
No. Observations:                  49   AIC:                             149.5
Df Residuals:                      43   BIC:                             160.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

,Condition,Effect,p_value
61,right_acc_Controls,C(Drug)[T.Placebo],0.007127
100,left_insula_Controls,Avoidance_centered,0.011464
52,right_amygdala_Patients,Avoidance_centered,0.026721
13,left_vmpfc_Patients,C(Drug)[T.Placebo],0.030061
106,right_insula_Controls,Avoidance_centered,0.045906
...,...,...,...
47,right_insula_Patients,demo_age,0.937662
58,right_vmpfc_Patients,Avoidance_centered,0.952469
53,right_amygdala_Patients,demo_age,0.954535
34,right_hippocampus_Patients,Avoidance_centered,0.957675


in group specific dataset, \
for Patients: gender in lh-insula \
        avoidance in rh-amy and lh-vmpfc
for Controls: drug effect in rh_ACC, and avoidance in lh-insula

In [52]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == contrasts_CSR, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'ROI': roi,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [53]:
results_list = []
# extinction, in each gender
for g in df_current['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {g}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Gender': g,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for right_acc Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.201
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     2.468
Date:                Wed, 26 Mar 2025   Prob (F-statistic):             0.0452
Time:                        09:33:25   Log-Likelihood:                -71.865
No. Observations:                  55   AIC:                             155.7
Df Residuals:                      49   BIC:                             167.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

,Condition,Gender,Effect,p_value
2,right_acc_Female,Female,C(Drug)[T.Placebo],0.005755
111,right_amygdala_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.028666
35,right_hippocampus_Female,Female,demo_age,0.035776
117,right_vmpfc_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.038575
3,right_acc_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.039243
...,...,...,...,...
64,right_acc_Male,Male,Avoidance_centered,0.924503
47,right_insula_Female,Female,demo_age,0.934550
93,right_hippocampus_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.935756
62,right_acc_Male,Male,C(Drug)[T.Placebo],0.939739


in Gender specific data subset, \
for Female: \
    drug effect in rh-acc, \
    group * drug in rh-acc
    age in rh-hip \
    avoidance in lh-vmpfc \
for Male: \
    age in rh-vmpfc \
    group * drun in rh-amy \
    avoidance(marginal) in lh_amygdala

In [54]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df_current['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'extinction results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin(contrasts_of_interest))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Gender': g,
                    'Group': group,
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for right_acc Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                 -0.068
Method:                 Least Squares   F-statistic:                    0.4076
Date:                Wed, 26 Mar 2025   Prob (F-statistic):              0.749
Time:                        09:33:27   Log-Likelihood:                -40.457
No. Observations:                  29   AIC:                             88.91
Df Residuals:                      25   BIC:                             94.38
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

,Gender,Group,Condition,Effect,p_value
117,Male,Patients,right_vmpfc_Patients_Male,C(Drug)[T.Placebo],0.003739
41,Female,Controls,right_acc_Controls_Female,C(Drug)[T.Placebo],0.015117
23,Female,Patients,right_hippocampus_Patients_Female,demo_age,0.032579
89,Male,Patients,left_vmpfc_Patients_Male,C(Drug)[T.Placebo],0.050277
113,Male,Patients,right_amygdala_Patients_Male,C(Drug)[T.Placebo],0.052653
...,...,...,...,...,...
97,Male,Patients,left_acc_Patients_Male,C(Drug)[T.Placebo],0.939157
26,Female,Patients,left_insula_Patients_Female,Avoidance_centered,0.941343
42,Female,Controls,right_acc_Controls_Female,Avoidance_centered,0.942018
138,Male,Controls,left_acc_Controls_Male,Avoidance_centered,0.960813


in group * gender specific data subset:\
avoidance in lh-vmpfc for Male controls \
drug in rh-acc for Female controls \
age in rh-hip for Female patients \
avoidance in rh-vmpfc for Male controls \

In [55]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                col='group',
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_group.png', bbox_inches='tight')
plt.close()
#plt.show()

In [56]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [57]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_group_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [58]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast'] == contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in Extinction', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'{plot_dir}/extinction_plot_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

right_acc
left_hippocampus
left_vmpfc
left_amygdala
left_acc
right_hippocampus
left_insula
right_insula
right_amygdala
right_vmpfc
